## Gene Selection for Network Construction

In [23]:
import scanpy as sc
import os
import pandas as pd
import numpy as np
import loompy as lp
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()
data_dir = '/data/'

### Filtering adata

In [98]:
adata_file = data_dir + '/Anndata.h5ad' 

In [99]:
adata = sc.read(adata_file)

In [2]:
# PT_address = data_dir + 'to_cancer_L6/to_cancer_Lineage6_new.csv'
PT_address = data_dir + 'to_entero_L3/to_entero_Lineage3_new.csv'
Diff_PT = pd.read_csv(PT_address,index_col=0)
PT_cells = Diff_PT.dropna().index
# Diff_PT

In [63]:
adata_t = adata[PT_cells]
adata_t.shape, adata.var_names, adata.obs_names

In [60]:
adata = adata_t

In [61]:
adata.X.transpose().shape # Gene symbol X Cell id

(31053, 1235)

In [20]:
# INITIAL PREPROCESSING-> SAVE LOOM FILE
# 
f_loom_path_scenic = data_dir + "input.loom"
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

### STEP 1: Gene regulatory network inference, and generation of co-expression modules

Phase Ia: GRN inference using the GRNBoost2 algorithm

In [22]:
f_tfs = data_dir + 'mm_tf.txt'

In [15]:
!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 20

In [16]:
adjacencies = pd.read_csv(data_dir +"adj.csv", index_col=False, sep=',')

In [ ]:
adjacencies.head()

### STEP 2~3: Regulon prediction aka cisTarget from CLI

In [ ]:
import glob
# ranking databases
f_db_glob = "*.feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "/data/motifs-v9-nr.mgi-m0.001-o0.0.tbl"

In [ ]:
f_db_names

In [ ]:
!pyscenic ctx adj.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 20

### STEP 4: Cellular enrichment (aka AUCell) from CLI

In [ ]:
import numpy as np
nGenesDetectedPerCell = np.sum(adata.X>0, axis=1)
percentiles = np.quantile(nGenesDetectedPerCell,[.01, .05, .10, .50, 1])
print(percentiles)

In [ ]:
# path to pyscenic output
f_pyscenic_output = "output.loom"

!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 20

In [ ]:
# collect SCENIC AUCell output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()
auc_mtx.to_csv('auc_mtx.csv')

### Gene selection: regulon_specificity_scores

In [79]:
from pyscenic.rss import regulon_specificity_scores

In [3]:
# auc_mtx = pd.read_csv("/data/to_cancer_L6/auc_mtx.csv",index_col=0)
auc_mtx = pd.read_csv("/data/to_entero_L3/auc_mtx.csv",index_col=0)
# auc_mtx.shape, auc_mtx.iloc[:5,:5]

In [4]:
# Data_add='/data/to_cancer_L6/to_cancer_L6new_normalizedgene_expression.csv'
Data_add='/data/to_entero_L3new_normalizedgene_expression.csv'

data = pd.read_csv(Data_add,index_col=0)
var_genes = set((data.std()[data.std()>0.1]).index)
mean_genes = set((data.std()[data.mean()>0.05]).index)
var_genes = (var_genes & mean_genes)
len(var_genes)

In [102]:
cellAnnot = Diff_PT.dropna()
cellAnnot = pd.concat([cellAnnot, pd.DataFrame(adata[PT_cells,].obs.seurat_clusters)],axis=1)
cellAnnot

,PT,seurat_clusters
ACTGATGCACGACGAA-1,0.000000,0
TACCTTATCTAAGCCA-1,0.000000,1
AGAGTGGAGACAAGCC-1,0.081017,1
AATCGGTTCACTTACT-1,0.370780,2
CGAGCCAAGAGATGAG-1,0.377285,2
...,...,...
AGATTGCTCTCGAGTA-1,20.680929,5
CATCAGAAGTAGGTGC-1,20.702917,5
TGGCCAGCAAACAACA-1,20.703051,5
TCTGAGAGTTCCTCCA-1,20.710960,5


In [103]:
import numpy as np
from collections import Counter
p25 = np.quantile(Diff_PT.dropna(),0.25)
p75 = np.quantile(Diff_PT.dropna(),0.75)

cellAnnot.loc[:,'q2'] = ['Unknown' for _ in range(cellAnnot.shape[0])]

# to_entero_L3
cellAnnot.loc[[ (('2' == x) | ('6' == x)) & (y<= p25) for x,y in zip(cellAnnot.seurat_clusters,cellAnnot.PT)],'q2'] = '26'
cellAnnot.loc[[ (('13' == x) | ('5' == x)) & (y>= p75) for x,y in zip(cellAnnot.seurat_clusters,cellAnnot.PT)],'q2'] = '135'

# to_cancer_L6
# cellAnnot.loc[[ (('2' == x) | ('2' == x)) & (y<= p25) for x,y in zip(cellAnnot.seurat_clusters,cellAnnot.PT)],'q2'] = '2'
# cellAnnot.loc[[ (('7' == x) | ('7' == x)) & (y>= p75) for x,y in zip(cellAnnot.seurat_clusters,cellAnnot.PT)],'q2'] = '7'
print(Counter(cellAnnot.q2))


Counter({'Unknown': 1464, '26': 658, '135': 519})


In [104]:
rss_cellType = regulon_specificity_scores( auc_mtx, cellAnnot['q2'] )
rss_cellType

,1810024B03Rik(+),2010315B03Rik(+),9030624G23Rik(+),AI987944(+),AU041133(+),AW146154(+),Acaa1a(+),Acaa1b(+),Aco1(+),Ahr(+),...,Zfp868(+),Zfp931(+),Zfp950(+),Zfp959(+),Zfp961(+),Zfx(+),Zmiz1(+),Zscan21(+),Zscan22(+),Zxdc(+)
Unknown,0.492032,0.557161,0.265257,0.493993,0.388472,0.256667,0.231339,0.252443,0.493780,0.560684,...,0.312752,0.513030,0.562083,0.484945,0.385596,0.538014,0.498845,0.353378,0.258817,0.272040
26,0.378067,0.359931,0.249553,0.339673,0.357164,0.236908,0.222423,0.209214,0.301124,0.390817,...,0.248570,0.372341,0.357730,0.339210,0.305128,0.375680,0.337822,0.375821,0.312099,0.203548
135,0.337213,0.374368,0.265475,0.331944,0.259201,0.278992,0.277277,0.546497,0.458981,0.334475,...,0.306669,0.345796,0.358158,0.353520,0.304430,0.367692,0.444624,0.275239,0.200079,0.396027


In [105]:
# rss_cellType_toC

In [85]:
p = 0.9
genes = set(rss_cellType.T['2'][rss_cellType.T['2'] >= np.quantile(rss_cellType.T['2'], p)].index)
toC_stem = set([x.split('(')[0] for x in genes]) & var_genes
print(toC_stem, len(toC_stem))

{'Cebpb', 'Pole4', 'Gtf2f1', 'Nr2c2', 'Nr3c1', 'Smarcb1', 'Tfdp2', 'Polr3a', 'Mlxipl', 'Myb', 'Tfdp1', 'Hmgb3', 'Hnf4a', 'Esrra', 'Isx', 'Mthfd1', 'Cdx1', 'Atf1', 'Hnf4g'} 19


In [87]:
p = 0.9
genes = set(rss_cellType.T['7'][rss_cellType.T['7'] >= np.quantile(rss_cellType.T['7'], p)].index)
toC_cancer = set([x.split('(')[0] for x in genes]) & var_genes
print(toC_cancer, len(toC_cancer))

{'Usf1', 'Zfp444', 'Tcf4', 'Zfp652', 'Hoxb7', 'Foxa3', 'Foxq1', 'Ltf', 'Hoxb9', 'Tead2', 'Tgif1', 'Hoxb8', 'Sp5', 'Npdc1', 'Etv3', 'Nfe2l3'} 16


In [106]:
# rss_cellType_toE

In [107]:
p = 0.92
genes = set(rss_cellType_toE.T['26'][rss_cellType_toE.T['26'] >= np.quantile(rss_cellType_toE.T['26'], p)].index)
toE_stem = set([x.split('(')[0] for x in genes]) & var_genes
print(toE_stem, len(toE_stem))

{'Myc', 'Nelfe', 'Smc3', 'Hes1', 'E2f5', 'Pole4', 'Hdac2', 'Bmyc', 'Pole3', 'Ezh2', 'Sox9', 'Elf2', 'Ehf', 'Hes6', 'Zfp704', 'Tfdp2', 'Gmeb1'} 17


In [108]:
p = 0.92
genes = set(rss_cellType_toE.T['135'][rss_cellType_toE.T['135'] >= np.quantile(rss_cellType_toE.T['135'], p)].index)
toE_entero = set([x.split('(')[0] for x in genes]) & var_genes
print(toE_entero, len(toE_entero))

{'Hif1a', 'Relb', 'Kdm7a', 'Gata5', 'Aco1', 'Mafb', 'Irf7', 'Rara', 'Maf', 'Zmiz1', 'Esrra', 'Nuak2', 'Ppargc1a', 'Klf4', 'Creb3l2', 'Max', 'Acaa1b', 'Tbx3', 'Mxi1', 'Bhlhe40'} 20
